In [1]:
import requests
import requests_cache
import pandas as pd
import itertools
from bs4 import BeautifulSoup
import pandas as pd
import math
import time

from bokeh.models import ColumnDataSource, LabelSet,HoverTool
from bokeh.plotting import figure, show, output_file
from bokeh.sampledata.periodic_table import elements

In [2]:
requests_cache.install_cache('craigslist_cache')

In [3]:
from data_function import *
from cleaning_function import *

# Data Preparation 

### Web Scraping 

In [4]:
df = get_all_search('cto', 'civic', 1999, 2000)
df.head()

,location,price,title
0,(watsonville),2000,2000 Honda civic Ex
1,(san jose north),3000,Very clean Honda Civic: 2000: 146k: $3k
2,(san jose east),2200,2000 Honda Civic Lx
3,(sunnyvale),1300,Honda civic 2000
4,(vallejo / benicia),2100,1999 Honda Civic Sedan


### Data Cleaning

In [6]:
df = df.replace(np.nan, 'Unknown', regex=True)
df.location = clean_location(df.location)
df.location = list(map(lambda x: x if len(x) != 0 else 'Unknown', df.location)) # Place Unknown if the location is empty
df.head()


C:\Users\Jim\Desktop\Github\usedcaranalysis\cleaning_function.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  location_list[i] = re.sub(r'[()]', "", location_list[i])
C:\Users\Jim\Desktop\Github\usedcaranalysis\cleaning_function.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  location_list[i] = location_list[i].strip(' ')


,location,price,title
0,Watsonville,2000,2000 Honda civic Ex
1,San jose north,3000,Very clean Honda Civic: 2000: 146k: $3k
2,San jose east,2200,2000 Honda Civic Lx
3,Sunnyvale,1300,Honda civic 2000
4,Vallejo / benicia,2100,1999 Honda Civic Sedan


# Data Visualization

In [7]:
location=df["location"]
price = df["price"]
low = min(price)
high = max(price)
palette = ["#053061", "#2166ac", "#4393c3", "#92c5de", "#d1e5f0",
           "#f7f7f7", "#fddbc7", "#f4a582", "#d6604d", "#b2182b", "#67001f"]
price_inds = [int(10*(x-low)/(high-low)) for x in price] 
df['price_colors'] = [palette[i] for i in price_inds]


TITLE = "Price v.s. Location"
TOOLS = "hover,pan,wheel_zoom,box_zoom,reset,save"

p = figure(y_range=list(set(location)), tools=TOOLS, toolbar_location="above", logo="grey", plot_width=1200, title=TITLE)
hover = p.select(dict(type=HoverTool))
hover.tooltips = [
    ("price", "@price"),
    ("location:", "@location"),
    ("title", "@title"),
]

source = ColumnDataSource(df)

p.circle('price', 'location', size=15, source=source, color='price_colors', line_width=3)

output_file("Data Visualization.html")

show(p)